In [2]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(pd.options.display.max_rows)
pd.set_option("display.max_columns", None)  # print all columns of a dataframe; we can replace "None" by any number if we want to print a certain number of columns
pd.set_option("max_colwidth", None)   # Set the Column Width.
pd.set_option("display.max_rows", 100) 

2.0.3
60


In [ ]:
#################  Size of the Original Sales files  ################# 
# Total: 830400 rows × 569 columns
# Germany: 330955 rows × 569 columns
# United kingdom: 260410 rows × 569 columns
# France: 239035 rows × 569 columns

# Combine datasets in 3 countries

In [ ]:
### Load single sales data ###
# Mac
df_uk_sales = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/raw data/sales/2024_enhanced_uk.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')
df_fr_sales = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/raw data/sales/2024_enhanced_fr.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')
df_ge_sales = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/raw data/sales/2024_enhanced_ge.csv', encoding='unicode_escape',  engine='python')

# rename the columns 'Country/Territory-Name' or 'ï»¿c' to 'country'
df_uk_sales.rename(columns={'Country/Territory-Name':'country'}, inplace=True)
df_fr_sales.rename(columns={'Country/Territory-Name':'country'}, inplace=True)
df_ge_sales.rename(columns={'ï»¿c':'country'}, inplace=True)

# concatenate the three dataframes horizontally
df = pd.concat([df_uk_sales, df_fr_sales, df_ge_sales],axis=0, ignore_index=True)

# Delete rows with zero sales in year 2015-2023

In [ ]:
# drop rows with no sales in years 2015 to 2023
df = df[ (df.loc[:,'2015/01':'2023/12']!=0).any(axis=1) ]  
# drop columns '1981/01' to '2014/12' and '2024/01' to '2024/06':
df = df.drop(df.loc[:, '1981/01':'2014/12'].columns, axis = 1)
df = df.drop(df.loc[:, '2024/01':'2024/06'].columns, axis = 1)

# Aggregate monthly sales to annual sales

In [ ]:
# aggregate from monthly sales to annuals sales
for year in range(2015, 2024):
    month_columns = [f"{year}/{month:02d}" for month in range(1, 13)]  # this create a list of column names for the 12 months of the year, useful skill!
    df[str(year)] = df[month_columns].sum(axis=1, skipna=True)         # Sum the monthly values for each year, handling missing columns
df.drop(columns=df.filter(like='/'), inplace=True)                     # Drop the original monthly columns        # columns=df_s.filter(like='/') pick the columns that contain '/', useful skill!

In [ ]:
### Save to a csv file
df.to_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/processed data/sales/2015_2023_sales_all.csv', index=False) 

# Aggregate duplicates (too granular, not needed anymore!!)

In [ ]:
### Show the number of sales in each year before aggregate duplicates
#df['2019'].sum()   # 8,126,958
#df['2020'].sum()   # 6,195,737
#df['2021'].sum()   # 5,924,511
#df['2022'].sum()   # 5,793,956
#df['2023'].sum()   # 6,519,213

# show the number of sales in 2023 for each country
#df[df['country']== 'Germany']['2023'].sum()   # 2,844,609
#df[df['country']== 'United Kingdom']['2023'].sum()   # 1,903,054
#df[df['country']== 'France']['2023'].sum()   # 1,771,550


# check the column types of df
df.dtypes

# remove the empty space at the beginning and the end of the string
df['Price. (local)'] = df['Price. (local)'].str.strip() 
df['Price. (EUR)'] = df['Price. (EUR)'].str.strip()

# for column 'Price. (local)', 'Price. (EUR), replace 'Unspecified' with NaN
df['Price. (local)'] = df['Price. (local)'].replace('Unspecified', np.nan)
df['Price. (EUR)'] = df['Price. (EUR)'].replace('Unspecified', np.nan)

# change the column 'Price. (local)' to float
df['Price. (local)'] = df['Price. (local)'].astype(float)
df['Price. (EUR)'] = df['Price. (EUR)'].astype(float)


### check for potential duplicates

# create the list of column for grouping
df_select = df.drop(df.loc[:, 'Price. (local)' : '2023'].columns, axis = 1)  
df['duplicate'] = df_select.groupby(df_select.columns.tolist(), dropna=False).transform('size')     # option: grouped by selected columns: list(df_select.columns[1:])
df_sort = df.sort_values(by = ['duplicate'] + df_select.columns.tolist(), ascending = [False] + [True]*(len(df_select.columns)))  # sort by 'duplicate_count' in descending order

# observation: the same car model with different(updated) prices are treated as different observations


# aggregate the duplicates
select_columns = df.columns[df.columns.get_loc('country'):df.columns.get_loc('Wheelbase')+1]    # good way to select columns
df_agg = df.groupby(select_columns.tolist(), dropna=False).agg({'Price. (local)': 'mean', 'Price. (EUR)': 'mean', '2015':'sum', '2016':'sum', '2017':'sum', '2018':'sum', '2019':'sum', '2020':'sum', '2021':'sum', '2022':'sum', '2023':'sum'})
df_agg = df_agg.reset_index() 

# aggregate the duplicates without grouping by 'country'
select_columns = df.columns[df.columns.get_loc('Make Group'):df.columns.get_loc('Wheelbase')+1]
df_agg = df.groupby(select_columns.tolist(), dropna=False).agg({'Price. (local)': 'mean', 'Price. (EUR)': 'mean', '2015':'sum', '2016':'sum', '2017':'sum', '2018':'sum', '2019':'sum', '2020':'sum', '2021':'sum', '2022':'sum', '2023':'sum'})
df_agg = df_agg.reset_index() 

# Aggregate the rows by: Parent-Brand-Body type-Fuel type.

In [45]:
# Mac
df = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/processed data/sales/2015_2023_sales_all.csv', low_memory=False)
# Windows
#df = pd.read_csv('C:/Users/uctpql0/OneDrive - University College London/Cars data/data/processed data/sales/2015_2023_sales_all.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')

In [49]:
### Miscellaneous
# Rename Columns
df.columns = df.columns.str.lower() 
df = df.rename(columns={'price. (local)': 'price_local'})     
df = df.rename(columns={'price. (eur)': 'price_eur'})          
df = df.rename(columns={'gross vehicle weight ': 'weight'})
df = df.rename(columns={'engine (ccm)': 'power'})
df = df.rename(columns={'engine (hp)': 'horsepower'})
df = df.rename(columns={'make group': 'parent'})
df = df.rename(columns={'make': 'brand'})
# Change the data type of columns
df = df[df['price_eur']!='Unspecified'] # drop rows with 'Unspecified' in 'price_eur'.   This reduce the unique parent number from 68 to 49!
df['price_local'] = df['price_local'].astype(int)  
df['price_eur'] = df['price_eur'].astype(int)

In [50]:
### 1. aggregate body group into 5 classes: Car(Coupe, Sedan, Hatchback), SUV, Wagon, MPV, Convertible 
# rename values in 'body group' such that 'Sedan', 'Coupe', 'Hatchback' are classified as 'Car', the rest stay the same
df['body group'] = df['body group'].str.lower()
df['body group'] = df['body group'].str.rstrip()
df['body group'] = df['body group'].str.lstrip()
df['body group'] = df['body group'].replace({'sedan': 'car', 'coupe': 'car', 'hatchback': 'car'})

In [51]:
### 2. aggregate fuel type into 4 classes: (BEV)pure electric, PHEV(plug-in electric), Hybrid, ICE
# rename values in ''fuel type'
df['fuel type'] = df['fuel type'].str.lower()
df['fuel type'] = df['fuel type'].str.rstrip()
df['fuel type'] = df['fuel type'].str.lstrip()
# Internal combustion engine (ICE)
df['fuel type'] = df['fuel type'].replace('diesel','ice')
df['fuel type'] = df['fuel type'].replace('petrol','ice')
# Hybrid
df['fuel type'] = df['fuel type'].replace('hev/petr','hybrid')
df['fuel type'] = df['fuel type'].replace('hev/dsl','hybrid')
df['fuel type'] = df['fuel type'].replace('mev/petrol','hybrid')
df['fuel type'] = df['fuel type'].replace('mev/diesel','hybrid')
df['fuel type'] = df['fuel type'].replace('hev series pet','hybrid')
df['fuel type'] = df['fuel type'].replace('hev/petr.unsp','hybrid')
df['fuel type'] = df['fuel type'].replace('hev/dsl.unsp.','hybrid')
# Plug-in Hybrid Electric Vehicle (PHEV)
df['fuel type'] = df['fuel type'].replace('hev/dsl.plugin','phev')
df['fuel type'] = df['fuel type'].replace('hev/petr.plugin','phev')
df['fuel type'] = df['fuel type'].replace('electric w. rex','phev')
# Battery Electric Vehicle (BEV)
df['fuel type'] = df['fuel type'].replace('electric w/orex','bev')

In [54]:
### 3. group the data by 'country', 'parent', 'brand', 'body group', 'fuel type'
# Group the data
df = df.groupby(['country','parent', 'brand', 'body group', 'fuel type']).agg({
    'power': 'mean',      # for car characteristics, take the mean of different trims
    'horsepower': 'mean',
    'weight': 'mean', 
    'price_local': 'mean', 
    'price_eur':'mean',    
    '2015': 'sum',
    '2016' : 'sum',
    '2017' : 'sum',
    '2018' : 'sum',
    '2019' : 'sum',
    '2020' : 'sum',
    '2021' : 'sum',
    '2022' : 'sum',
    '2023' : 'sum'   # for sales data, sum the sales across different trims
}).round(2).copy()
df = df.reset_index()   # Reset the index to make the groupby columns back to columns

In [ ]:
### 4. only keep rows with 'body group' in ['car', 'suv', 'wagon', 'mpv', 'convertible'] and 'fuel type' in ['ICE', 'Hybrid', 'PHEV', 'BEV']
df = df[df['body group'].isin(['car', 'suv', 'wagon', 'mpv', 'convertible'])]
df = df[df['fuel type'].isin(['ice', 'hybrid', 'phev', 'bev'])]
# one of the above selections reduce the unique parent number from 49 to 43! 

In [23]:
### 5. Modify parent name to be recognizable
# load the second sheet of the excel file, 
dm  = pd.read_excel('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Parent company.xlsx', sheet_name='Sales')
# transform the values in 'make group' to lower case
df['parent'] = df['parent'].str.lower()
# replace the name of the column 'parent' in df by the 'modified name' in dm according to the 'original name' in dm
df['parent'] = df['parent'].replace(dict(zip(dm['original name'], dm['modified name'])))

In [24]:
### 6. Modify the brand name 
# rename the values in 'brand': 'Opel' to 'Opel-Vauxhall', 'Vauxhall' to 'Opel-Vauxhall'
df['brand'] = df['brand'].replace('Opel', 'Opel-Vauxhall')
df['brand'] = df['brand'].replace('Vauxhall', 'Opel-Vauxhall')

Number of unique values in parent, brand, body group and fuel type:
1) 43 parents
2) 83 brands
3) 5 body groups
4) 4 fuel types
3) 401 unique combinations of parent, brand, body type and fuel type

In [ ]:
# save the aggregate sales data to a csv file
df.to_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/processed data/sales/2015_2023_sales_aggregate.csv', index=False)
# load the aggregate sales data
df = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/data/processed data/sales/2015_2023_sales_aggregate.csv', low_memory=False)